In [161]:
import pandas as pd
import numpy as np
import seaborn as sns
import re

In [162]:
df=pd.read_csv('train.csv')

In [163]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes,15000
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,63500
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,7850


In [164]:
# df['model_year']=df['model_year'].apply(lambda x:2024-x)

In [165]:
q25=df['milage'].quantile(0.25)
q75=df['milage'].quantile(0.75)
iqr=q75-q25
up=q75+1.5*iqr
down=q25-1.5*iqr
# df['milage']=df['milage'].apply(lambda x: up if x>up else x)

In [166]:
fuel_type = df['engine'].apply(lambda x:None if re.search(r"(Gasoline|Diesel|Electric|Flex Fuel)", x) is None else 1)
df["fuel_type_is_missing"] = pd.isnull(fuel_type)

In [167]:
df['model'].value_counts()

F-150 XLT             780
M3 Base               682
X5 xDrive35i          615
F-150 Lariat          496
Mustang GT Premium    466
                     ... 
Suburban 2500           1
TLX Tech                1
Rogue Sport S           1
A8 4.0                  1
S-10 LS                 1
Name: model, Length: 1827, dtype: int64

In [168]:
df['model_year']

0        2018
1        2007
2        2009
3        2022
4        2001
         ... 
54268    2017
54269    2015
54270    2013
54271    2023
54272    2021
Name: model_year, Length: 54273, dtype: int64

In [169]:
df.describe()

,id,model_year,milage,price
count,54273.000000,54273.000000,54273.000000,5.427300e+04
mean,27136.000000,2015.091979,72746.175667,3.921844e+04
std,15667.409917,5.588909,50469.490448,7.282634e+04
min,0.000000,1974.000000,100.000000,2.000000e+03
25%,13568.000000,2012.000000,32268.000000,1.550000e+04
50%,27136.000000,2016.000000,66107.000000,2.800000e+04
75%,40704.000000,2019.000000,102000.000000,4.500000e+04
max,54272.000000,2024.000000,405000.000000,2.954083e+06


In [170]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54273 entries, 0 to 54272
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    54273 non-null  int64 
 1   brand                 54273 non-null  object
 2   model                 54273 non-null  object
 3   model_year            54273 non-null  int64 
 4   milage                54273 non-null  int64 
 5   fuel_type             54273 non-null  object
 6   engine                54273 non-null  object
 7   transmission          54273 non-null  object
 8   ext_col               54273 non-null  object
 9   int_col               54273 non-null  object
 10  accident              54273 non-null  object
 11  clean_title           54273 non-null  object
 12  price                 54273 non-null  int64 
 13  fuel_type_is_missing  54273 non-null  bool  
dtypes: bool(1), int64(4), object(9)
memory usage: 5.4+ MB


In [171]:
df['horsepower']=df['engine'].apply(lambda x:float(x[:4]) if x[5:7]=='HP' else None)
label='horsepower'
df[label + "_is_missing"] = pd.isnull(df['horsepower'])
# Fill missing numeric values with median 
df[label] = df['horsepower'].fillna(df['horsepower'].median())

In [172]:
q25=df['horsepower'].quantile(0.25)
q75=df['horsepower'].quantile(0.75)
iqr=q75-q25
up=q75+1.5*iqr
down=q25-1.5*iqr
# df['horsepower']=df['horsepower'].apply(lambda x: up if x>up else x)

In [173]:
df['cylinder_config']=df['engine'].apply(lambda engine_str: "V" if "V" in engine_str else "Straight" if "Straight" in engine_str else None)
label='cylinder_config'
df[label + "_is_missing"] = pd.isnull(df['cylinder_config'])
# Fill missing numeric values with median 
df[label] = df['cylinder_config'].fillna("idontknow0")

In [174]:
df['cylinder_config']

0                 V
1          Straight
2        idontknow0
3          Straight
4                 V
            ...    
54268    idontknow0
54269    idontknow0
54270             V
54271             V
54272    idontknow0
Name: cylinder_config, Length: 54273, dtype: object

In [175]:
cylinder_config=pd.get_dummies(df['cylinder_config'])
df=pd.concat([df,cylinder_config],axis=1)

In [176]:
import re
v1=df['engine'].str.extract(r'\s(\d+)\s')

In [177]:
v1=pd.DataFrame({'index':range(0,54273),'number':v1[0]})
# v1[0]

In [178]:
v2=df['engine'].str.extract(r'\s\w(\d+)\s')

In [179]:
v2=pd.DataFrame({'index':range(0,54273),'number':v2[0]})

In [180]:
newv=pd.merge(v1, v2, on='index', how='outer')

In [181]:
newv.head()

,index,number_x,number_y
0,0,NaN,6
1,1,6,NaN
2,2,8,NaN
3,3,6,NaN
4,4,NaN,6


In [182]:
df['v']=newv['number_x'].combine_first(newv['number_y'])

In [183]:
label='v'
df[label + "_is_missing"] = pd.isnull(df['v'])
df[label] = df['v'].fillna('6')

In [184]:
df['v']=df['v'].astype(int)

In [185]:
l1=df['engine'].str.extract(r'\s(\d+\.\d+)\w\s')
l1=pd.DataFrame({'index':range(0,54273),'number':l1[0]})

In [186]:
l2=df['engine'].str.extract(r'\b(\d+\.\d+)\sLiter\b')
l2=pd.DataFrame({'index':range(0,54273),'number':l2[0]})

In [187]:
newv=pd.merge(l1, l2, on='index', how='outer')
df['L']=newv['number_x'].combine_first(newv['number_y'])


In [188]:
# df['L']=df['L'].fillna('3.72')
# df['L'] = df['L'].astype(float)
label='L'
df[label + "_is_missing"] = pd.isnull(df['L'])
df[label] = df['L'].fillna('3.72')
df['L']=df['L'].astype(float)

In [189]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,...,horsepower_is_missing,cylinder_config,cylinder_config_is_missing,Straight,V,idontknow0,v,v_is_missing,L,L_is_missing
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,...,False,V,False,0,1,0,6,False,3.5,False
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,...,False,Straight,False,1,0,0,6,False,3.0,False
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,...,False,idontknow0,True,0,0,1,8,False,4.2,False
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,...,False,Straight,False,1,0,0,6,False,3.0,False
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,...,False,V,False,0,1,0,6,False,3.8,False


In [190]:
df['transmission'].value_counts()

A/T                                                   16757
8-Speed A/T                                            7287
Transmission w/Dual Shift Mode                         6454
6-Speed A/T                                            5944
6-Speed M/T                                            3618
7-Speed A/T                                            3415
10-Speed A/T                                           2187
8-Speed Automatic                                      1189
5-Speed A/T                                            1185
9-Speed A/T                                            1073
4-Speed A/T                                             764
5-Speed M/T                                             718
CVT Transmission                                        692
10-Speed Automatic                                      642
6-Speed Automatic                                       516
Automatic                                               441
M/T                                     

In [191]:
# df['tranmission_no']=df['transmission'].str.extract(r'(\d+)')
def extract_transmission_features(transmission_str):
    if 'A/T' in transmission_str or 'Automatic' in transmission_str or 'Auto' in transmission_str or 'automatic' in transmission_str or 'auto' in transmission_str:
        transmission_type = 'A/T'
        num_gears = re.search(r'(\d+)-Speed', transmission_str)
        num_gears = int(num_gears.group(1)) if num_gears else None
        transmission_features = transmission_str.replace(str(num_gears) + '-Speed A/T', '').strip()
    elif 'M/T' in transmission_str or 'Manual' in transmission_str or 'manual'in transmission_str:
        transmission_type = 'M/T'
        num_gears = re.search(r'(\d+)-Speed', transmission_str)
        num_gears = int(num_gears.group(1)) if num_gears else None
        transmission_features = transmission_str.replace(str(num_gears) + '-Speed M/T', '').strip()
    elif 'Dual' in transmission_str or 'dual' in transmission_str:
        transmission_type = 'Dual'
        num_gears = 0
        transmission_features = transmission_str.strip()
    else:
        transmission_type = 'Unknown'
        num_gears = 0
        transmission_features = 'Unknown'
    return transmission_type, num_gears,transmission_features

In [192]:
df[['transmission_type', 'num_gears','transmission_features']]=df['transmission'].apply(extract_transmission_features).apply(pd.Series)

In [193]:
# df['num_gears']=df['num_gears'].fillna(1)
# df['num_gears']=df['num_gears'].astype(int)
label='num_gears'
df[label + "_is_missing"] = pd.isnull(df['num_gears'])
df[label] = df['num_gears'].fillna(df['num_gears'].median())
df['num_gears']=df['num_gears'].astype(int)

In [194]:
label='transmission_features'
df[label + "_is_missing"] = pd.isnull(df['transmission_features'])
df[label] = df['transmission_features'].fillna("idontknow1")

In [195]:
label='transmission_type'
df[label + "_is_missing"] = pd.isnull(df['transmission_type'])
df[label] = df['transmission_type'].fillna("idontknow2")

In [196]:
arr=df['transmission_features'].apply(lambda x:sorted([i+"_trans" for i in re.findall(r'\b[a-zA-Z]+\b',x)] ))
colset=[]
for i in range(54273):
    for j in arr[i]:
        colset.append(j)
transdata=list(set(colset))
print(transdata)

['DCT_trans', 'Speed_trans', 'Shift_trans', 'Automatic_trans', 'T_trans', 'w_trans', 'Manual_trans', 'Electronically_trans', 'M_trans', 'Overdrive_trans', 'CVT_trans', 'Mode_trans', 'Controlled_trans', 'Transmission_trans', 'Auto_trans', 'with_trans', 'O_trans', 'Dual_trans', 'A_trans', 'Unknown_trans']


In [197]:
nottransdata=[i+"_is_missing" for i in transdata]
arr1=arr.apply(lambda x:[i+"_is_missing" for i in list(set(transdata)-set(x))])
for i in nottransdata:
    df[i]=0
for i in range(54273):
    for j in arr1[i]:
        if j in nottransdata:
            df.loc[i,j]=1

In [198]:
for i in transdata:
    df[i]=0
for i in range(54273):
    for j in arr[i]:
        if j in transdata:
            df.loc[i,j]=1

In [199]:
df.columns

Index(['id', 'brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine',
       'transmission', 'ext_col', 'int_col', 'accident', 'clean_title',
       'price', 'fuel_type_is_missing', 'horsepower', 'horsepower_is_missing',
       'cylinder_config', 'cylinder_config_is_missing', 'Straight', 'V',
       'idontknow0', 'v', 'v_is_missing', 'L', 'L_is_missing',
       'transmission_type', 'num_gears', 'transmission_features',
       'num_gears_is_missing', 'transmission_features_is_missing',
       'transmission_type_is_missing', 'DCT_trans_is_missing',
       'Speed_trans_is_missing', 'Shift_trans_is_missing',
       'Automatic_trans_is_missing', 'T_trans_is_missing',
       'w_trans_is_missing', 'Manual_trans_is_missing',
       'Electronically_trans_is_missing', 'M_trans_is_missing',
       'Overdrive_trans_is_missing', 'CVT_trans_is_missing',
       'Mode_trans_is_missing', 'Controlled_trans_is_missing',
       'Transmission_trans_is_missing', 'Auto_trans_is_missing',
       'with

In [200]:
acci=pd.get_dummies(df['accident'])
df=pd.concat([df,acci],axis=1)

In [201]:
acci=pd.get_dummies(df['transmission_type'])
df=pd.concat([df,acci],axis=1)

In [202]:
# int_col=pd.get_dummies(df['int_col'])
# df=pd.concat([df,int_col],axis=1)
# df['int_col']=df['int_col'].astype('category').cat.codes
# encoder1 = LabelEncoder()
# encoder1.fit(df['int_col'])

# # Transform training data
# df['int_col'] = encoder1.transform(df['int_col'])
intdict={}
for i in range(len(list(set(df['int_col'])))):
    intdict[list(set(df['int_col']))[i]]=i
# print(intdict)
df['int_col']=df['int_col'].replace(intdict)

In [203]:
clean_title=pd.get_dummies(df['clean_title'])
df=pd.concat([df,clean_title],axis=1)

In [204]:
# ext_col=pd.get_dummies(df['ext_col'])
# df=pd.concat([df,ext_col],axis=1)
# df['int_col']=df['int_col'].astype('category').cat.codes
# encoder2 = LabelEncoder()
# encoder2.fit(df['ext_col'])

# # Transform training data
# df['ext_col'] = encoder2.transform(df['ext_col'])
extdict={}
for i in range(len(list(set(df['ext_col'])))):
    extdict[list(set(df['ext_col']))[i]]=i
print(intdict)
df['ext_col']=df['ext_col'].replace(extdict)
# df['ext_col']

{'Charcoal': 0, 'Dark Gray': 1, 'Camel Leather': 2, 'Hotspur Hide': 3, 'BLACK': 4, 'Ebony / Pimento': 5, 'Black / Graphite': 6, 'Jet Black': 7, 'Red / Black': 8, 'Oyster W/Contrast': 9, 'Canberra Beige/Black': 10, 'Red': 11, 'Charcoal Black': 12, 'Platinum': 13, 'Almond Beige': 14, 'Cappuccino': 15, 'Cobalt Blue': 16, 'Beige': 17, 'Linen': 18, 'Black / Express Red': 19, 'Ebony / Ebony Accents': 20, 'Black': 21, 'Navy Pier': 22, 'Ivory / Ebony': 23, 'Dark Ash': 24, 'AMG Black': 25, 'Caramel': 26, 'Titan Black / Quarzit': 27, 'Black/Gun Metal': 28, 'Yellow': 29, 'Mocha': 30, 'Nero': 31, 'Graystone': 32, 'Silver': 33, 'Walnut': 34, 'Dark Auburn': 35, 'Oyster/Black': 36, 'Adrenaline Red': 37, 'Medium Ash Gray': 38, 'Deep Garnet': 39, 'Light Titanium': 40, 'Black w/Red Stitching': 41, 'Nougat Brown': 42, 'Light Gray': 43, 'Deep Chestnut': 44, 'Charles Blue': 45, 'Beluga Hide': 46, 'Shara Beige': 47, 'Giallo Taurus / Nero Ade': 48, 'Tupelo': 49, 'Sandstone': 50, 'Mistral Gray / Raven': 51, '

In [205]:
fuel_type=pd.get_dummies(df['fuel_type'])
df=pd.concat([df,fuel_type],axis=1)

In [206]:
brand=pd.get_dummies(df['brand'])
df=pd.concat([df,brand],axis=1)

In [207]:
# df['model']=df['model'].astype('category').cat.codes
def extract_model_features(model):
    parts = model.split()
    base_model = parts[0] if parts else model
    model_variant = parts[1] if len(parts) > 1 else None
    additional_features = ''.join(parts[2:]) if len(parts) > 2 else None
    return base_model, model_variant, additional_features
df[['base_model', 'model_variant', 'additional_features']] =df['model'].apply(lambda x: pd.Series(extract_model_features(x)))

In [208]:
label='base_model'
df[label + "_is_missing"] = pd.isnull(df['base_model'])
df[label] = df['base_model'].fillna("idontknow")

In [209]:
label='model_variant'
df[label + "_is_missing"] = pd.isnull(df['base_model'])
df[label] = df['base_model'].fillna("idontknow")

In [210]:
label='additional_features'
df[label + "_is_missing"] = pd.isnull(df['additional_features'])
df[label] = df['additional_features'].fillna("idontknow")

In [211]:
base_modeldict={}
for i in range(len(list(set(df['base_model'])))):
    base_modeldict[list(set(df['base_model']))[i]]=i
df['base_model']=df['base_model'].replace(base_modeldict)

model_variantdict={}
for i in range(len(list(set(df['model_variant'])))):
    model_variantdict[list(set(df['model_variant']))[i]]=i
df['model_variant']=df['model_variant'].replace(model_variantdict)

additional_featuresdict={}
for i in range(len(list(set(df['additional_features'])))):
    additional_featuresdict[list(set(df['additional_features']))[i]]=i
df['additional_features']=df['additional_features'].replace(additional_featuresdict)


# df['model_variant']=df['model_variant'].astype('category').cat.codes
# df['additional_features']=df['additional_features'].astype('category').cat.codes

In [212]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,...,Tesla,Toyota,Volkswagen,Volvo,base_model,model_variant,additional_features,base_model_is_missing,model_variant_is_missing,additional_features_is_missing
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,97,97,...,0,0,0,0,303,303,437,False,False,True
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,47,21,...,0,0,0,0,26,26,437,False,False,True
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,171,17,...,0,0,0,0,6,6,437,False,False,True
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,190,104,...,0,0,0,0,232,232,437,False,False,True
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,258,21,...,0,0,0,0,324,324,437,False,False,True


In [213]:
df.drop(columns=['brand','fuel_type','accident','clean_title','id','model'])

,model_year,milage,engine,transmission,ext_col,int_col,price,fuel_type_is_missing,horsepower,horsepower_is_missing,...,Tesla,Toyota,Volkswagen,Volvo,base_model,model_variant,additional_features,base_model_is_missing,model_variant_is_missing,additional_features_is_missing
0,2018,74349,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,97,97,11000,False,375.0,False,...,0,0,0,0,303,303,437,False,False,True
1,2007,80000,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,47,21,8250,False,300.0,False,...,0,0,0,0,26,26,437,False,False,True
2,2009,91491,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,171,17,15000,False,300.0,False,...,0,0,0,0,6,6,437,False,False,True
3,2022,2437,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,190,104,63500,False,335.0,False,...,0,0,0,0,232,232,437,False,False,True
4,2001,111000,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,258,21,7850,False,200.0,False,...,0,0,0,0,324,324,437,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54268,2017,29000,445.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,8-Speed A/T,258,104,29000,False,445.0,False,...,0,0,0,0,390,390,437,False,False,True
54269,2015,94634,220.0HP 2.0L 4 Cylinder Engine Flex Fuel Capab...,6-Speed A/T,47,21,6500,False,220.0,False,...,0,0,0,0,41,41,125,False,False,False
54270,2013,40989,420.0HP 3.6L V6 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,258,21,18950,False,420.0,False,...,0,0,0,0,473,473,437,False,False,True
54271,2023,1518,4.0L H6 24V GDI DOHC,8-Speed Automatic with Auto-Shift,33,104,194965,True,310.0,True,...,0,0,0,0,244,244,421,False,False,False


In [214]:
arr=df['engine'].apply(lambda x:sorted(re.findall(r'\b[a-zA-Z]+\b',x)))
# arr[0]
colset=[]
for i in range(54273):
    for j in arr[i]:
        colset.append(j)

In [215]:
enginedata=['AH',
 'Battery',
 'Capability',
 'Cylinder',
 'DDI',
 'DOHC',
 'Diesel',
 'Electric',
 'Engine',
 'Flat',
 'Flex',
 'Flexible',
 'GDI',
 'GTDI',
 'Gas',
 'Gasoline',
 'Hybrid',
 'Hydrogen',
 'In',
 'Intercooled',
 'Liter',
 'MPFI',
 'Mild',
 'Motor',
 'OHV',
 'PDI',
 'Plug',
 'Range',
 'Rotary',
 'SC',
 'SOHC',
 'Standard',
 'Supercharged',
 'System',
 'TFSI',
 'TSI',
 'Turbo',
 'Twin',
 'ULEV',
 'VTEC',
 'engine']

In [216]:
for i in enginedata:
    df[i]=0
for i in range(54273):
    for j in arr[i]:
        if j in enginedata:
            df.loc[i,j]=1

In [217]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,...,SOHC,Standard,Supercharged,System,TFSI,TSI,Turbo,Twin,ULEV,VTEC
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,0,10-Speed A/T,97,97,...,0,0,0,0,0,0,0,0,0,0
1,1,BMW,335 i,2007,80000,Gasoline,0,6-Speed M/T,47,21,...,0,0,0,0,0,0,0,0,0,0
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,0,6-Speed A/T,171,17,...,0,0,0,0,0,0,0,0,0,0
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,0,Transmission w/Dual Shift Mode,190,104,...,0,0,0,0,0,0,0,0,0,0
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,0,A/T,258,21,...,0,0,0,0,0,0,0,0,0,0


In [218]:
df=df.dropna(axis=1)

In [219]:
df=df.drop(columns=['brand','fuel_type','accident','clean_title','id','model','transmission_type','transmission_features','cylinder_config'],axis=1)

In [220]:
df.head()

,model_year,milage,engine,transmission,ext_col,int_col,price,fuel_type_is_missing,horsepower,horsepower_is_missing,...,SOHC,Standard,Supercharged,System,TFSI,TSI,Turbo,Twin,ULEV,VTEC
0,2018,74349,0,10-Speed A/T,97,97,11000,False,375.0,False,...,0,0,0,0,0,0,0,0,0,0
1,2007,80000,0,6-Speed M/T,47,21,8250,False,300.0,False,...,0,0,0,0,0,0,0,0,0,0
2,2009,91491,0,6-Speed A/T,171,17,15000,False,300.0,False,...,0,0,0,0,0,0,0,0,0,0
3,2022,2437,0,Transmission w/Dual Shift Mode,190,104,63500,False,335.0,False,...,0,0,0,0,0,0,0,0,0,0
4,2001,111000,0,A/T,258,21,7850,False,200.0,False,...,0,0,0,0,0,0,0,0,0,0


In [221]:
target=df['price']
df=df.drop(columns=['transmission'])

In [222]:
# df['noyear']=pd.cut(df['model_year'],9,labels=[str(i) for i in range(9)])

In [223]:
df=df.drop(columns=['price',])

In [224]:
df.columns.to_list()

['model_year',
 'milage',
 'engine',
 'ext_col',
 'int_col',
 'fuel_type_is_missing',
 'horsepower',
 'horsepower_is_missing',
 'cylinder_config_is_missing',
 'Straight',
 'V',
 'idontknow0',
 'v',
 'v_is_missing',
 'L',
 'L_is_missing',
 'num_gears',
 'num_gears_is_missing',
 'transmission_features_is_missing',
 'transmission_type_is_missing',
 'DCT_trans_is_missing',
 'Speed_trans_is_missing',
 'Shift_trans_is_missing',
 'Automatic_trans_is_missing',
 'T_trans_is_missing',
 'w_trans_is_missing',
 'Manual_trans_is_missing',
 'Electronically_trans_is_missing',
 'M_trans_is_missing',
 'Overdrive_trans_is_missing',
 'CVT_trans_is_missing',
 'Mode_trans_is_missing',
 'Controlled_trans_is_missing',
 'Transmission_trans_is_missing',
 'Auto_trans_is_missing',
 'with_trans_is_missing',
 'O_trans_is_missing',
 'Dual_trans_is_missing',
 'A_trans_is_missing',
 'Unknown_trans_is_missing',
 'DCT_trans',
 'Speed_trans',
 'Shift_trans',
 'Automatic_trans',
 'T_trans',
 'w_trans',
 'Manual_trans',
 '

In [225]:
!pip3 install catboost

In [226]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = MinMaxScaler()
stanscaler=StandardScaler()
# dft=stanscaler.fit_transform(df)
X_train, X_test, y_train, y_test = train_test_split(df, target, test_size=0.1, random_state=42)

In [227]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
model = RandomForestRegressor(n_jobs =-1, random_state = 42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
rmse1 = np.sqrt(mean_squared_error(y_test, y_pred))
print("The Root mean square is " + str(rmse1))

The Root mean square is 53834.82053027416


In [228]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
catboost=CatBoostRegressor(loss_function='RMSE',depth=5,random_seed=73,iterations=2500,learning_rate=0.01,verbose=False)
# catboost=RandomForestRegressor( n_estimators=50, random_state=42)
catboost.fit(X_train, y_train)

In [229]:
from sklearn.metrics import mean_squared_error
y_pred = catboost.predict(X_test)
rmse1 = np.sqrt(mean_squared_error(y_test, y_pred))
print("The Root mean square is " + str(rmse1))

The Root mean square is 42449.05696642335


In [68]:
import random
df1=pd.read_csv('test.csv')
df1.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,54273,Mercedes-Benz,E-Class E 350,2014,73000,Gasoline,302.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,White,Beige,None reported,Yes
1,54274,Lexus,RX 350 Base,2015,128032,Gasoline,275.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,Silver,Black,None reported,Yes
2,54275,Mercedes-Benz,C-Class C 300,2015,51983,Gasoline,241.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Blue,White,None reported,Yes
3,54276,Land,Rover Range Rover 5.0L Supercharged Autobiogra...,2018,29500,Gasoline,518.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,White,At least 1 accident or damage reported,Yes
4,54277,BMW,X6 xDrive40i,2020,90000,Gasoline,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes


In [69]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36183 entries, 0 to 36182
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            36183 non-null  int64 
 1   brand         36183 non-null  object
 2   model         36183 non-null  object
 3   model_year    36183 non-null  int64 
 4   milage        36183 non-null  int64 
 5   fuel_type     36183 non-null  object
 6   engine        36183 non-null  object
 7   transmission  36183 non-null  object
 8   ext_col       36183 non-null  object
 9   int_col       36183 non-null  object
 10  accident      36183 non-null  object
 11  clean_title   36183 non-null  object
dtypes: int64(3), object(9)
memory usage: 3.3+ MB


In [70]:
df1.isnull().sum()

id              0
brand           0
model           0
model_year      0
milage          0
fuel_type       0
engine          0
transmission    0
ext_col         0
int_col         0
accident        0
clean_title     0
dtype: int64

In [71]:
df1.replace('-','idontknow',inplace=True)

In [72]:
q25=df1['milage'].quantile(0.25)
q75=df1['milage'].quantile(0.75)
iqr=q75-q25
up=q75+1.5*iqr
down=q25-1.5*iqr
# df1[df1['milage']>up]
# df1['milage']=df1['milage'].apply(lambda x: up if x>up else x)

In [73]:
fuel_type = df1['engine'].apply(lambda x:None if re.search(r"(Gasoline|Diesel|Electric|Flex Fuel)", x) is None else 1)
df1["fuel_type_is_missing"] = pd.isnull(fuel_type)

In [74]:
df1['horsepower']=df1['engine'].apply(lambda x:float(x[:4]) if x[5:7]=='HP' else None)
label='horsepower'
df1[label + "_is_missing"] = pd.isnull(df1['horsepower'])
# Fill missing numeric values with median 
df1[label] = df1['horsepower'].fillna(df1['horsepower'].median())

In [75]:
q25=df1['horsepower'].quantile(0.25)
q75=df1['horsepower'].quantile(0.75)
iqr=q75-q25
up=q75+1.5*iqr
down=q25-1.5*iqr
# df1['horsepower']=df1['horsepower'].apply(lambda x: up if x>up else x)

In [76]:
df1['cylinder_config']=df1['engine'].apply(lambda engine_str: "V" if "V" in engine_str else "Straight" if "Straight" in engine_str else None)
label='cylinder_config'
df1[label + "_is_missing"] = pd.isnull(df1['cylinder_config'])
df1[label] = df1['cylinder_config'].fillna("idontknow0")
cylinder_config=pd.get_dummies(df1['cylinder_config'])
df1=pd.concat([df1,cylinder_config],axis=1)

In [77]:
v1=df1['engine'].str.extract(r'\s(\d+)\s')
v1=pd.DataFrame({'index':range(0,36183),'number':v1[0]})
v2=df1['engine'].str.extract(r'\s\w(\d+)\s')
v2=pd.DataFrame({'index':range(0,36183),'number':v2[0]})
newv=pd.merge(v1, v2, on='index', how='outer')
newv.head()
df1['v']=newv['number_x'].combine_first(newv['number_y'])
# df1['v']=df1['v'].fillna('6')
# df1['v'] = df1['v'].astype(int)
l1=df1['engine'].str.extract(r'\s(\d+\.\d+)\w\s')
l1=pd.DataFrame({'index':range(0,36183),'number':l1[0]})
l2=df1['engine'].str.extract(r'\b(\d+\.\d+)\sLiter\b')
l2=pd.DataFrame({'index':range(0,36183),'number':l2[0]})
newv=pd.merge(l1, l2, on='index', how='outer')
df1['L']=newv['number_x'].combine_first(newv['number_y'])
# df1['L']=df1['L'].fillna('3.72')
# df1['L'] = df1['L'].astype(float)
label='v'
df1[label + "_is_missing"] = pd.isnull(df1['v'])
df1[label] = df1['v'].fillna('6')
df1['v']=df1['v'].astype(int)
label='L'
df1[label + "_is_missing"] = pd.isnull(df1['L'])
df1[label] = df1['L'].fillna('3.72')
df1['L']=df1['L'].astype(float)

In [78]:
# df1['tranmission_no']=df1['transmission'].str.extract(r'(\d+)')[0]
df1[['transmission_type', 'num_gears','transmission_features']]=df1['transmission'].apply(extract_transmission_features).apply(pd.Series)

In [79]:
label='transmission_type'
df1[label + "_is_missing"] = pd.isnull(df1['transmission_type'])
df1[label] = df1['transmission_type'].fillna("idontknow2")
label='transmission_features'
df1[label + "_is_missing"] = pd.isnull(df1['transmission_features'])
df1[label] = df1['transmission_features'].fillna("idontknow1")
label='num_gears'
df1[label + "_is_missing"] = pd.isnull(df1['num_gears'])
df1[label] = df1['num_gears'].fillna(df1['num_gears'].median())
df1['num_gears']=df1['num_gears'].astype(int)

In [80]:
df1[['transmission_type', 'num_gears','transmission_features']]

,transmission_type,num_gears,transmission_features
0,A/T,6,A/T
1,A/T,8,
2,A/T,7,
3,Dual,0,Transmission w/Dual Shift Mode
4,A/T,8,
...,...,...,...
36178,A/T,10,
36179,A/T,4,
36180,M/T,5,
36181,A/T,10,


In [81]:
# df1['num_gears']=df1['num_gears'].fillna(1)
# df1['num_gears']=df1['num_gears'].astype(int)
# df1['transmission_features']=df1['transmission_features'].replace({"":"idontknow"})
# df1['transmission_features']=df1['transmission_features'].fillna("idontknow")
acci=pd.get_dummies(df1['transmission_type'])
df1=pd.concat([df1,acci],axis=1)

In [82]:
df1[['transmission_type', 'num_gears','transmission_features']]

,transmission_type,num_gears,transmission_features
0,A/T,6,A/T
1,A/T,8,
2,A/T,7,
3,Dual,0,Transmission w/Dual Shift Mode
4,A/T,8,
...,...,...,...
36178,A/T,10,
36179,A/T,4,
36180,M/T,5,
36181,A/T,10,


In [83]:
arr=df1['transmission_features'].apply(lambda x:sorted([i+"_trans" for i in re.findall(r'\b[a-zA-Z]+\b',x)]))
colset=[]
for i in range(36183):
    for j in arr[i]:
        colset.append(j)
transdata=list(set(colset))
print(transdata)

['DCT_trans', 'Speed_trans', 'Shift_trans', 'Automatic_trans', 'T_trans', 'w_trans', 'Manual_trans', 'Electronically_trans', 'M_trans', 'Overdrive_trans', 'CVT_trans', 'Mode_trans', 'Controlled_trans', 'Transmission_trans', 'Auto_trans', 'with_trans', 'O_trans', 'Dual_trans', 'A_trans', 'Unknown_trans']


In [ ]:
arr1=arr.apply(lambda x:[i+"_is_missing" for i in list(set(transdata)-set(x))])
for i in nottransdata:
    df1[i]=0
for i in range(54273):
    for j in arr1[i]:
        if j in nottransdata:
            df1.loc[i,j]=1

In [84]:
for i in transdata:
    df1[i]=0
for i in range(36183):
    for j in arr[i]:
        if j in transdata:
            df1.loc[i,j]=1

In [85]:
brand=pd.get_dummies(df1['brand'])
df1=pd.concat([df1,brand],axis=1)
fuel_type=pd.get_dummies(df1['fuel_type'])
df1=pd.concat([df1,fuel_type],axis=1)
# ext_col=pd.get_dummies(df1['ext_col'])
# df1=pd.concat([df1,ext_col],axis=1)
clean_title=pd.get_dummies(df1['clean_title'])
d1f=pd.concat([df1,clean_title],axis=1)
# int_col=pd.get_dummies(df1['int_col'])
# df1=pd.concat([df1,int_col],axis=1)
df1['int_col']=df1['int_col'].apply(lambda x:intdict[x] if x in intdict else random.randint(502, 1000))
df1['ext_col']=df1['ext_col'].apply(lambda x:extdict[x] if x in extdict else random.randint(502, 1000))
acci1=pd.get_dummies(df1['accident'])
df1=pd.concat([df1,acci1],axis=1)

In [86]:
# df1['model']=df1['model'].astype('category').cat.codes
df1[['base_model', 'model_variant', 'additional_features']] =df1['model'].apply(lambda x: pd.Series(extract_model_features(x)))
label='additional_features'
df1[label + "_is_missing"] = pd.isnull(df1['additional_features'])
df1[label] = df1['additional_features'].fillna("idontknow")
label='model_variant'
df1[label + "_is_missing"] = pd.isnull(df1['base_model'])
df1[label] = df1['base_model'].fillna("idontknow")
label='base_model'
df1[label + "_is_missing"] = pd.isnull(df1['base_model'])
df1[label] = df1['base_model'].fillna("idontknow")
df1['base_model']=df1['base_model'].apply(lambda x:base_modeldict[x] if x in base_modeldict else random.randint(502, 1000))
df1['model_variant']=df1['model_variant'].apply(lambda x:model_variantdict[x] if x in model_variantdict else random.randint(502, 1000))
df1['additional_features']=df1['additional_features'].apply(lambda x:additional_featuresdict[x] if x in additional_featuresdict else random.randint(502, 1000))

In [87]:
# df1['noyear']=pd.cut(df1['model_year'],9,labels=[str(i) for i in range(9)])
# df1['noyear']=df1['noyear'].astype(int)
# df1['d']=df1['milage']*df1['v']
df1[['base_model', 'model_variant', 'additional_features']]

,base_model,model_variant,additional_features
0,4,4,453
1,401,401,228
2,94,94,455
3,409,409,374
4,390,390,437
...,...,...,...
36178,263,263,437
36179,295,295,384
36180,333,333,437
36181,284,284,437


In [88]:
df1=df1.drop(columns=['brand','fuel_type','accident','clean_title','id','transmission','model','transmission_type','transmission_features','cylinder_config'],axis=1)

In [89]:
arr=df1['engine'].apply(lambda x:sorted(re.findall(r'\b[a-zA-Z]+\b',x)))
# arr[0]
colset=[]
for i in range(36183):
    for j in arr[i]:
        colset.append(j)

In [90]:
enginedata=['AH',
 'Battery',
 'Capability',
 'Cylinder',
 'DDI',
 'DOHC',
 'Diesel',
 'Electric',
 'Engine',
 'Flat',
 'Flex',
 'Flexible',
 'GDI',
 'GTDI',
 'Gas',
 'Gasoline',
 'Hybrid',
 'Hydrogen',
 'In',
 'Intercooled',
 'Liter',
 'MPFI',
 'Mild',
 'Motor',
 'OHV',
 'PDI',
 'Plug',
 'Range',
 'Rotary',
 'SC',
 'SOHC',
 'Standard',
 'Supercharged',
 'System',
 'TFSI',
 'TSI',
 'Turbo',
 'Twin',
 'ULEV',
 'VTEC',
 'engine']

In [91]:
for i in enginedata:
    df1[i]=0
for i in range(36183):
    for j in arr[i]:
        if j in enginedata:
            df1.loc[i,j]=1

In [92]:
df1['Cylinder'][1]

1

In [93]:
df1=df1.dropna(axis=1)

In [94]:
# catboost.predict(df1)
# df1.shape
df1=df1.drop(['Saab', 'smart'],axis=1)

In [95]:
df1['Yes']=1
df1['Maybach']=0
df1['Plymouth']=0

In [96]:
# df1=df1.drop(['d'],axis=1)
# set(df.columns.tolist())-set(df1.columns.tolist())
set(df1.columns.tolist())-set(df.columns.tolist())

set()

In [97]:
result=pd.DataFrame({'id':pd.read_csv('test.csv')['id'],'price':catboost.predict(df1)})
# result=pd.DataFrame({'id':pd.read_csv('test.csv')['id'],'price':model.predict(df1)})

In [98]:
result.to_csv('out.csv',index=False)

In [99]:
catboost.predict(df1)

array([22109.5837382 , 19585.34794902, 29501.78323441, ...,
       13550.96577595, 55109.86964195, 18612.77933847])

In [100]:
[i for i in df.columns.to_list() if '_is_missing' in i]

['fuel_type_is_missing',
 'horsepower_is_missing',
 'cylinder_config_is_missing',
 'v_is_missing',
 'L_is_missing',
 'num_gears_is_missing',
 'transmission_features_is_missing',
 'transmission_type_is_missing',
 'base_model_is_missing',
 'model_variant_is_missing',
 'additional_features_is_missing']